<a href="https://colab.research.google.com/github/DhrumilPrajapati03/Segmentation_model_using_U-net/blob/main/Ahmedabad_sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing on Ahmedabad's sentinel-2 Dataset

#### Installing libs


In [ ]:
!pip install patchify
!pip install segmentation-models
!pip install tensorflow

#### converting from jp2 to tiff

In [ ]:
import os
import rasterio
from rasterio.shutil import copy as rio_copy

input_folder = "/content/drive/MyDrive/S2A_MSIL1C_20151120T054122_N0500_R005_T43QBF_20231008T183202.SAFE/S2B_MSIL1C_20201128T054159_N0500_R005_T43QBF_20230409T052646.SAFE/S2B_MSIL1C_20201128T054159_N0500_R005_T43QBF_20230409T052646.SAFE/GRANULE/L1C_T43QBF_A019479_20201128T054158/IMG_DATA"  # The folder with .jp2 files
output_folder = "/content/drive/MyDrive/S2A_MSIL1C_20151120T054122_N0500_R005_T43QBF_20231008T183202.SAFE/S2B_MSIL1C_20201128T054159_N0500_R005_T43QBF_20230409T052646.SAFE/S2B_MSIL1C_20201128T054159_N0500_R005_T43QBF_20230409T052646.SAFE/GRANULE/L1C_T43QBF_A019479_20201128T054158/tiff files02"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith(".jp2"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename.replace(".jp2", ".tif"))

        with rasterio.open(input_path) as src:
            rio_copy(src, output_path, driver='GTiff')
            print(f"Converted {filename} to .tif")

#### Checking Dimensions of both images

In [ ]:
# prompt: give me script to find the dimensions of the two tif images
import rasterio
# Paths to your two input images
image1_path = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/2015/amd_2015.tif'
image2_path = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/2020/amd_2020.tif'

with rasterio.open(image1_path) as src1:
    print(f"Image 1 dimensions: {src1.width} x {src1.height}")

with rasterio.open(image2_path) as src2:
    print(f"Image 2 dimensions: {src2.width} x {src2.height}")

#### patchifying both images and save in png format

In [ ]:
# prompt: give me code to do 64x64 patches of both images and sav in png form

from patchify import patchify

def create_patches(image_path, output_dir, patch_size=64):
    """Creates patches of a given size from an image and saves them as PNGs."""
    try:
        with rasterio.open(image_path) as src:
            image = src.read()
            patches = patchify(image, (image.shape[0], patch_size, patch_size), step=patch_size)

            # Iterate through patches
            patch_index = 0
            for i in range(patches.shape[0]):
                for j in range(patches.shape[1]):
                    for k in range(patches.shape[2]):
                        patch = patches[i, j, k, :, :]
                        patch_filename = os.path.join(output_dir, f'patch_{patch_index:04d}.png')

                        # Convert to PIL image and save
                        patch_image = Image.fromarray(patch.astype(np.uint8))
                        patch_image.save(patch_filename)

                        patch_index += 1

            print(f"✅ Patches created and saved in {output_dir}")

    except Exception as e:
        print(f"⚠️ Error processing {image_path}: {str(e)}")


# Example usage for image1 and image2
image1_path = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/2015/amd_2015.tif'
image2_path = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/2020/amd_2020.tif'

output_dir_image1 = '/content/drive/MyDrive/image1_patches'
output_dir_image2 = '/content/drive/MyDrive/image2_patches'

os.makedirs(output_dir_image1, exist_ok=True)
os.makedirs(output_dir_image2, exist_ok=True)

create_patches(image1_path, output_dir_image1)
create_patches(image2_path, output_dir_image2)


#### Counting total number of patches

In [ ]:
import os

def count_patches(folder_path):
  """Counts the number of patches in a given folder."""
  patch_count = 0
  for filename in os.listdir(folder_path):
    if filename.endswith(('.tif', '.png')):  # Adjust file extensions if needed
      patch_count += 1
  return patch_count

# Example usage:
folder1 = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2015'
folder2 = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2020'


total_patches = count_patches(folder1) + count_patches(folder2)
print(f"Total number of patches in both folders: {total_patches}")


#### plot first 5 patches

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Define the folder paths
folder_2015 = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2015'
folder_2020 = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2020'

# Function to load images from a folder and return the first n images
def load_images(folder_path, num_images=5):
    images = []
    for i, filename in enumerate(os.listdir(folder_path)):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Modify the extensions as per your image formats
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            images.append(image)
        if len(images) == num_images:
            break
    return images

# Load the first 5 patches from both folders
patches_2015 = load_images(folder_2015, num_images=5)
patches_2020 = load_images(folder_2020, num_images=5)

# Plot the images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Plot 2015 patches
for i, ax in enumerate(axes[0]):
    ax.imshow(patches_2015[i])
    ax.axis('off')  # Hide axes
    ax.set_title(f'Patch {i+1} (2015)')

# Plot 2020 patches
for i, ax in enumerate(axes[1]):
    ax.imshow(patches_2020[i])
    ax.axis('off')  # Hide axes
    ax.set_title(f'Patch {i+1} (2020)')

plt.tight_layout()
plt.show()

#### Plotting Before-after and predicted mask

In [ ]:
import numpy as np
from PIL import Image
from skimage import io
import segmentation_models as sm
from tensorflow import keras
import matplotlib.pyplot as plt

# ... (previous code for loading model and preprocessing) ...
# prompt: instead of test_triples i want to predict my images seperately, means i'll give two tif images and it should give predicted output

import numpy as np
from PIL import Image
from skimage import io
import segmentation_models as sm
from tensorflow import keras

# --- Load model ---
model_path = '/content/drive/MyDrive/U_NET Segmentation_model/model.keras'
from segmentation_models import Unet
model = Unet(backbone_name='resnet34', encoder_weights='imagenet', decoder_block_type='upsampling')
# Compile the model with the desired loss function
def custom_loss(y_true, y_pred):
    bce = keras.losses.BinaryCrossentropy()
    jaccard = sm.losses.JaccardLoss()
    return bce(y_true, y_pred) + jaccard(y_true, y_pred)
model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])
# Load the weights from the saved model file
model.load_weights(model_path)
# model = tf.keras.models.load_model(model_path)
print("✅ Model Loaded")

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

def preprocess_image(image_path):
    img = Image.open(image_path)
    img = np.array(img) / 255
    img = img.astype(np.float32)
    img = preprocess_input(img)
    return img

def predict_image_pair(image1_path, image2_path):
    # Preprocess images
    im1 = preprocess_image(image1_path)
    im2 = preprocess_image(image2_path)

    # Calculate the difference
    x = im1 - im2

    # Expand dimensions for model input
    x = np.expand_dims(x, axis=0)

    # Make prediction
    y_pred = model.predict(x)
    y_pred = y_pred[0]

    return y_pred


# Example usage
image1_path = "/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2015/patch_0_192.png"  # Replace with the actual path
image2_path = "/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2020/patch_0_192.png"  # Replace with the actual path

prediction = predict_image_pair(image1_path, image2_path)

# --- Display images ---
image1 = Image.open(image1_path)
image2 = Image.open(image2_path)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(image1)
axes[0].set_title("Image 1")

axes[1].imshow(image2)
axes[1].set_title("Image 2")

axes[2].imshow(prediction[:, :, 0], cmap='gray')  # Assuming prediction is in (H, W, C) format
axes[2].set_title("Predicted Change Mask")

plt.show()

#### Plotting first 10 patches from both folder along with their predicted mask


In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import io
import segmentation_models as sm
from tensorflow import keras

# --- Load model ---
model_path = '/content/drive/MyDrive/U_NET Segmentation_model/model.keras'
from segmentation_models import Unet
model = Unet(backbone_name='resnet34', encoder_weights='imagenet', decoder_block_type='upsampling')

# Compile the model with the desired loss function
def custom_loss(y_true, y_pred):
    bce = keras.losses.BinaryCrossentropy()
    jaccard = sm.losses.JaccardLoss()
    return bce(y_true, y_pred) + jaccard(y_true, y_pred)

model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

# Load the weights from the saved model file
model.load_weights(model_path)
print("✅ Model Loaded")

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# Preprocessing function
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = np.array(img) / 255
    img = img.astype(np.float32)
    img = preprocess_input(img)
    return img

# Prediction function for a pair of images
def predict_image_pair(image1_path, image2_path):
    # Preprocess images
    im1 = preprocess_image(image1_path)
    im2 = preprocess_image(image2_path)

    # Calculate the difference
    x = im1 - im2

    # Expand dimensions for model input
    x = np.expand_dims(x, axis=0)

    # Make prediction
    y_pred = model.predict(x)
    y_pred = y_pred[0]

    return y_pred

# Paths to the folders containing patches
folder_2015 = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2015/'
folder_2020 = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2020/'

# Function to load first n patches from the folders
def load_patches(folder_path, num_patches=10):
    patches = []
    for i, filename in enumerate(sorted(os.listdir(folder_path))[:num_patches]):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            patches.append(image_path)
    return patches

# Load the first 10 patches from both folders
patches_2015 = load_patches(folder_2015, num_patches=10)
patches_2020 = load_patches(folder_2020, num_patches=10)

# --- Plotting the patches with predicted masks ---
fig, axes = plt.subplots(10, 3, figsize=(15, 50))  # Adjusted for 10 rows

for i in range(10):
    image1_path = patches_2015[i]
    image2_path = patches_2020[i]

    # Get prediction for this pair of patches
    prediction = predict_image_pair(image1_path, image2_path)

    # Load original images
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)

    # Display the images and predicted mask
    axes[i, 0].imshow(image1)
    axes[i, 0].set_title(f"Image 1 - Patch {i+1}")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(image2)
    axes[i, 1].set_title(f"Image 2 - Patch {i+1}")
    axes[i, 1].axis('off')

    axes[i, 2].imshow(prediction[:, :, 0], cmap='gray')  # Assuming prediction is in (H, W, C) format
    axes[i, 2].set_title(f"Predicted Mask - Patch {i+1}")
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

#### Plotting first 20 patches and predicted masks

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import io
import segmentation_models as sm
from tensorflow import keras

# --- Load model ---
model_path = '/content/drive/MyDrive/U_NET Segmentation_model/model.keras'
from segmentation_models import Unet
model = Unet(backbone_name='resnet34', encoder_weights='imagenet', decoder_block_type='upsampling')

# Compile the model with the desired loss function
def custom_loss(y_true, y_pred):
    bce = keras.losses.BinaryCrossentropy()
    jaccard = sm.losses.JaccardLoss()
    return bce(y_true, y_pred) + jaccard(y_true, y_pred)

model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

# Load the weights from the saved model file
model.load_weights(model_path)
print("✅ Model Loaded")

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# Preprocessing function
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = np.array(img) / 255
    img = img.astype(np.float32)
    img = preprocess_input(img)
    return img

# Prediction function for a pair of images
def predict_image_pair(image1_path, image2_path):
    # Preprocess images
    im1 = preprocess_image(image1_path)
    im2 = preprocess_image(image2_path)

    # Calculate the difference
    x = im1 - im2

    # Expand dimensions for model input
    x = np.expand_dims(x, axis=0)

    # Make prediction
    y_pred = model.predict(x)
    y_pred = y_pred[0]

    return y_pred

# Paths to the folders containing patches
folder_2015 = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2015/'
folder_2020 = '/content/drive/MyDrive/Ahmedabad_sentinel_dataset/patches_2020/'

# Function to load first n patches from the folders
def load_patches(folder_path, num_patches=20):
    patches = []
    for i, filename in enumerate(sorted(os.listdir(folder_path))[:num_patches]):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            patches.append(image_path)
    return patches

# Load the first 10 patches from both folders
patches_2015 = load_patches(folder_2015, num_patches=20)
patches_2020 = load_patches(folder_2020, num_patches=20)

# --- Plotting the patches with predicted masks ---
fig, axes = plt.subplots(20, 3, figsize=(15, 50))  # Adjusted for 10 rows

for i in range(20):
    image1_path = patches_2015[i]
    image2_path = patches_2020[i]

    # Get prediction for this pair of patches
    prediction = predict_image_pair(image1_path, image2_path)

    # Load original images
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)

    # Display the images and predicted mask
    axes[i, 0].imshow(image1)
    axes[i, 0].set_title(f"Patch {i+1} - Image 1")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(image2)
    axes[i, 1].set_title(f"Patch {i+1} - Image 2")
    axes[i, 1].axis('off')

    axes[i, 2].imshow(prediction[:, :, 0], cmap='gray')  # Assuming prediction is in (H, W, C) format
    axes[i, 2].set_title(f"Patch {i+1} - Predicted Mask")
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()